<a href="https://colab.research.google.com/github/sonalsrivas/Mathematical-Equation-Solver/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
'''       PROBLEMS
fails to recognize float numerals, mixed up, faulty methodology
->> resolve with nltk.word_tokenize(string) solutions


'''

''' The adhoc function meant to drive this program forward in absence of the necessary systems in place '''
def Convert_func(val,u1,u2):                      #Very speciifc, it is not going to bepart of the larger program
    #print(u1,u2)
    if u1=='kmph' and u2=='mps':
        return val*5/18
    elif u1=='C' and u2=='F':
        return val*9/5+32
    else:
      return 'exceedinglyYOLO'

''' This checks there is a 'in', 'to' or 'into' in the problem after the number signalling that it is a CONVERSION problem indeed '''
def intobtidnlast(problem,i,n):
  for j in range(i+1,n):
    if problem[j] in 'into':
      return True
  return False

def wordproblem(problem,numerical_data):                       #We will have list of lower case words of problem after numeral, unit recognition 
                                                  #and also the numerical data of the form [(id,numeral,unit,variable(optional))]
  ''' FOR COVERSION TYPE PROBLEMS '''
  if len(numerical_data)==1 and problem[0] in ['convert','what','find','change','determine'] and intobtidnlast(problem,numerical_data[0][0],len(problem)):
    convert_to=problem[-1]
    #We will have to map particular units in variables and we will have several unit synonyms pointing to the same variable
    #This is becoz in here we have the units as our variables themselves.
    
# Originally unit-UNIT associations were defined here
    numeral=numerical_data[0][1]; original_unit=numerical_data[0][2];
    val=numeral; u1=map_unit_UNIT[original_unit]; u2=map_unit_UNIT[convert_to]
    return Convert_func(val,u1,u2)
  #else:
    #print("YOLO")


map_unit_UNIT=dict()
map_UNIT_unit={'kmph':['kmph','km/h','km/hr'],'mps':['mps','m/s'],'C':['celsius','centigrade','c','degree'],'F':['f','fahrenheit'],
               'hour':['hour','hr'],'minute':['min','minute'],'second':['second','sec','s'],'day':['day'],'year':['year','yr'],
               'meter':['m','meter','metre'],'kilometer':['kilometer','km'],}
for vt in map_UNIT_unit:                                                         #this will save a little time
  for ut in map_UNIT_unit[vt]:
    map_unit_UNIT[ut]=vt
#print(map_unit_UNIT)

#map_unit_UNIT={'kmph':'kmph','km/h':'kmph','km/hr':'kmph',
#              'mps':'mps','m/s':'mps','m/min':'mpm',
#              'centigrade':'C','celsius':'C','degrees':'C','fahrenheit':'F'}


map_VAR_UNIT={'speed':{'kmph','mps'},'temperature':{'C','F'}, 'time':{'hour','minute','second','day','year'},'distance':['meter','kilometer'] }
#                                                                                                               !----------------!
#DONE #you could have done this with code 
map_UNIT_VAR=dict() #{'kmph':'speed','mps':'speed','C':'temperature','F':'temperature'}
for vt in map_VAR_UNIT:                                                         #this will save a little time
  for ut in map_VAR_UNIT[vt]:
    map_UNIT_VAR[ut]=vt
#print(map_UNIT_VAR)

map_VAR_var={'speed':{'velocity','speed'},'distance':{'distance','displacement','dist.'},'temparature':{'temparature'},'time':{'time'}}
map_var_VAR=dict()
for vt in map_VAR_var:                                                         #this will save a little time
  for ut in map_VAR_var[vt]:
    map_var_VAR[ut]=vt
#print(map_var_VAR)
#------------------------------------------------------------------------------------------------------------------------

#MODULAR
''' convert string to integer numeral or decimal numeral '''
def convert_numeral(word):
  print(word)
  if '.' in word:
    return float(word)
  else:
    return int(word)

'''removes punctuations
def remove_punctuations(string):
  punctuators='.,:;"#@!$^&*()_=<>?'       
  s=''
  for p in string:
    if p not in punctuators:
      s+=p
#    else:                                             #We are instead modifying any punctuation to appear as comma in order to signal separation of phrase
#      s+=' ,'
  return s
'''

''' cleans problem and splits in words '''
def listproblem(string):
  #s=remove_punctuations(string)
  problem=string.split()
  for p in range(len(problem)):
    if problem[p][-1]=='s' and problem[p][-2] not in 'aoui':
      problem[p]=problem[p][:-1]
    if problem[p][-1] in '.,?!':
      problem[p]=problem[p][:-1]                       #This is problematic USE NLTK MAYBE
    print("PROBLEM IN listproblem::",problem)
  return problem


'''produces a list containing ( numeral : unit )'''
def numeralunitList(problem):
  now=len(problem)
  numerical_data=[]                   #Of the form (id,numeral,unit,variable(optional))
  rr=0
  while rr<now-1:
    #print(rr,now)
    word=problem[rr]
    rr+=1
    #print(word)
    if word.isdigit():
      numerical_data.append((rr-1,convert_numeral(word),problem[rr]))
      rr+=1
    elif word.isalpha():
      continue
    else:
      first=True
      nocharencountered=True
      num=''
      for c in word:
        if first and c=='-':
          num+=c; first=False
        elif nocharencountered and (c.isdigit() or c=='.'):
          num+=c; first=False; nocharencountered=True
        elif c.isalpha():
          nocharencountered=False
          #print(word)
          numerical_data.append((rr-1,convert_numeral(num),word[word.index(c):]))
          break
        else:
          print("Kindly enter your word problem in a legible manner. Thank you.")
          exit(0)
  return numerical_data

def var_associations(problem,numerical_data):
  #check the words following and previuos to numeral is a keyword until 'and' or ',' or '.' appears
  ## numerical_data has structure of list with elements like (index, numeral, unit)
  ### we want another structure or will later modify numerical_data , that has (variable, numeral, unit) and maybe index as additional information
  #==> VarNumUnit=[]
  VarNumUnit=[]
  for j in numerical_data:
    k=1
    l=len(problem)
    f=0
    if j[0]>0 and problem[j[0]-1]=='in':
      VarNumUnit.append((map_UNIT_VAR[j[2]],j[1],j[2]))
      f=1
      continue
    while j[0]+k<l and problem[j[0]+k] not in ('and',','):
      if problem[j[0]+k] in map_var_VAR:
        VarNumUnit.append((map_var_VAR[problem[j[0]+k]],j[1],j[2]))
        f=1
        break
      k+=1
    k=1
    while f==0 and j[0]-k>-1 and problem[j[0]-k] not in ('and',','):
      if problem[j[0]-k] in map_var_VAR:
        VarNumUnit.append((map_var_VAR[problem[j[0]-k]],j[1],j[2]))
        f=1
        break
      k+=1
    if f==0:
      VarNumUnit.append((map_UNIT_VAR[j[2]],j[1],j[2]))
  return VarNumUnit

#---------------------------------------------THIS DRIVES THE UTILITY MODULES---------------------------------------------------------------------------
string=input(":Enter your word problem:\n").lower()
while string!='':
  problem=listproblem(string)
  numerical_data=numeralunitList(problem)
  print("Index, Numeral, Unit =>",numerical_data)
  Answer=wordproblem(problem, numerical_data)
  print(Answer)
  print("The problem =>",problem)
  
  Var_Num_Unit=var_associations(problem,numerical_data)
  print("Keyword, Numeral, Unit =>",Var_Num_Unit)
  
  string=input("\n:Enter your word problem:\n").lower()

:Enter your word problem:
If a sprinter runs a distance of 100 metres in 9.83 seconds, calculate his average speed in km/h.
PROBLEM IN listproblem:: ['if', 'a', 'sprinter', 'runs', 'a', 'distance', 'of', '100', 'metres', 'in', '9.83', 'seconds,', 'calculate', 'his', 'average', 'speed', 'in', 'km/h.']
PROBLEM IN listproblem:: ['if', 'a', 'sprinter', 'runs', 'a', 'distance', 'of', '100', 'metres', 'in', '9.83', 'seconds,', 'calculate', 'his', 'average', 'speed', 'in', 'km/h.']
PROBLEM IN listproblem:: ['if', 'a', 'sprinter', 'runs', 'a', 'distance', 'of', '100', 'metres', 'in', '9.83', 'seconds,', 'calculate', 'his', 'average', 'speed', 'in', 'km/h.']
PROBLEM IN listproblem:: ['if', 'a', 'sprinter', 'run', 'a', 'distance', 'of', '100', 'metres', 'in', '9.83', 'seconds,', 'calculate', 'his', 'average', 'speed', 'in', 'km/h.']
PROBLEM IN listproblem:: ['if', 'a', 'sprinter', 'run', 'a', 'distance', 'of', '100', 'metres', 'in', '9.83', 'seconds,', 'calculate', 'his', 'average', 'speed', 'in

KeyError: ignored